## Stock Tracker

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('Portfolio_Tracker')

In [3]:
c = conn.cursor()

In [21]:
stock_data = pd.read_csv('Stocks_Dataset.csv')
stock_data.to_sql('stock_data_raw',conn,if_exists='replace',index=False)

33106

In [19]:
c.execute('''create table stock (
  ID integer primary key autoincrement not null,
  tickersymbol varchar(10),
  companyname varchar(100)
)''')

In [24]:
c.execute('''insert into stock (tickersymbol,companyname)
select distinct symbol,company from stock_data_raw
''')

In [26]:
c.execute('''create table stockfact (
  stockid integer not null,
  date_time datetime,
  stockprice decimal(6,2),
  foreign key (stockid) references stock(id)
)''')

In [29]:
c.execute('''insert into stockfact(stockid,date_time,stockprice)
select distinct a.id,b.date,b.close_last from stock as a left join stock_data_raw as b on a.tickersymbol=b.symbol
''')

In [14]:
c.execute('''create table user (
  ID integer primary key autoincrement not null,
  FirstName char(50) not null,
  LastName char(50) not null,
  Email varchar(100) not null check(email like '%@%')
)''')

In [16]:
c.execute('''insert into user (FirstName,LastName,Email) values
('Lee','Johnson','lj@yahoo.com')
  '''
)

In [18]:
c.execute('''create table portfolio (
  ID integer primary key autoincrement not null,
  userid integer not null,
  foreign key (userid) references user(id)
)''')

In [30]:
c.execute('''insert into portfolio(userid)
select distinct id from user''')

In [55]:
c.execute('''create table stockportfolio (
  portfolioid integer not null,
  stockid integer not null,
  shares integer,
  foreign key (portfolioid) references portfolio (id),
  foreign key (stockid) references stocks(id)
)''')

In [56]:
c.execute('''insert into stockportfolio (portfolioid,stockid,shares)
select distinct a.id,b.id,1 from portfolio as a cross join stock as b where tickersymbol in ('INFY','JKS')
''')

In [57]:
c.execute('''select * from stockportfolio''')
colnames=[x[0] for x in c.description]
df = pd.DataFrame(c.fetchall(),columns=colnames)
print(df)

   portfolioid  stockid  shares
0            1        1       1
1            1       15       1
